In [ ]:
# Uncoment the following to work around an ocassisional bug in Colab:
# "A UTF-8 locale is required. Got ANSI_X3.4-1968"
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# Uncomment to clone and install autodoc from GitHub
# !pip uninstall -y autora-doc
# !git clone https://github.com/AutoResearch/autodoc.git
# !pip install "./autodoc[cuda,pipelines]"

# IMPORTANT: Please restart the runtime after running the above commands

In [ ]:
%load_ext autoreload
%autoreload 2
from autora.doc.runtime.predict_hf import Predictor, preprocess_code
from autora.doc.runtime.prompts import PROMPTS, PromptIds, PromptBuilder, SYS_GUIDES
from autora.doc.pipelines.metrics import eval_bleu_meteor, eval_semscore
from autora.doc.pipelines.main import eval_prompt, load_data

In [ ]:
model = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
pred = Predictor(model)

## Test generation for the variable declararion only

In [ ]:
TEST_VAR_CODE = """
iv = Variable(name="x", value_range=(0, 2 * np.pi), allowed_values=np.linspace(0, 2 * np.pi, 30))
dv = Variable(name="y", type=ValueType.REAL)
variables = VariableCollection(independent_variables=[iv], dependent_variables=[dv])
"""
LABEL = "The discovery problem is defined by a single independent variable $x \in [0, 2 \pi]$ and dependent variable $y$."

In [ ]:
def test(promptid, code, label):
    output = pred.predict(
        PROMPTS[promptid],
        [code],
        do_sample=0,
        max_new_tokens=100,
        temperature=0.05,
        top_k=10,
        num_ret_seq=1,
    )
    bleu, meteor = eval_bleu_meteor(output, [label])
    sem_score = eval_semscore(output, [label])

    for i, o in enumerate(output):
        print(
            f"{promptid}\n******* Output {i} ********. bleu={bleu}, meteor={meteor}, sem_score={sem_score}\n{o}\n*************\n"
        )

In [ ]:
# Zero shot test
test(PromptIds.AUTORA_VARS_ZEROSHOT, TEST_VAR_CODE, LABEL)

In [ ]:
# One shot test
test(PromptIds.AUTORA_VARS_ONESHOT, TEST_VAR_CODE, LABEL)

## One-shot generation for the complete code sample

In [ ]:
data_file = "../data/autora/data.jsonl"
# Use this path if running in Colab and cloning the repo
# data_file = "./autodoc/data/autora/data.jsonl"
inputs, labels = load_data(data_file)
# preprocessing removes comments, import statements and empty lines
inputs = [preprocess_code(i) for i in inputs]
INSTR = "Generate high-level, one or two paragraph documentation for the following experiment."
prompt = PromptBuilder(SYS_GUIDES, INSTR).add_example(f"{inputs[0]}", labels[0]).build()
print(prompt)

In [ ]:
eval_result = eval_prompt(data_file, pred, prompt, {"max_new_tokens": 800.0})
print(
    f"bleu={eval_result.bleu_score}, meteor={eval_result.meteor_score}, sem_score={eval_result.sem_score}\n{eval_result.predictions[0]}\n*************\n"
)